# Реализация базовых метрик

Ваша задача реализовать основные метрики которые мы сегодня обсудили и сравнить их на игрушечном примере.

In [4]:
UserRatings = {\
               "User1": [5,3,5,2,1,5,1,5,4,4],\
               "User2": [3,3,3,2,2,3,3,5,4,5],\
               "User3": [1,5,5,1,1,3,3,3,4,5]\
              }
UserRatings

{'User1': [5, 3, 5, 2, 1, 5, 1, 5, 4, 4],
 'User2': [3, 3, 3, 2, 2, 3, 3, 5, 4, 5],
 'User3': [1, 5, 5, 1, 1, 3, 3, 3, 4, 5]}

In [33]:
Model1Ratings = {\
               "User1": [1,3,4,2,2,4,3,4,4,4],\
               "User2": [1,3,4,2,1,3,2,5,5,5],\
               "User3": [1,3,5,1,3,4,4,5,4,3]\
              }

In [3]:
Model2Ratings = {\
               "User1": [5,4,5,4,2,3,2,5,5,5],\
               "User2": [4,4,3,3,1,2,2,3,3,5],\
               "User3": [3,5,5,2,4,2,2,4,4,4]\
              }

Выше вам даны оценки которые пользователи поставили фильмам и оценки которые предложили две модели. 

*Задача 1:* Вычислите MAE, MSE и RMSE для этих моделей. Какая модель лучше?

*Задача 2:* Реализуйте функцию Recomend(MarkList, N) которая по списку оценок выводит лист из N id фильмов, соответствующих лучшим оценкам. При равенстве оценок лучше считается вильм с меньшим id. id фильма соответствует индекcу в MarkList. 

*Задача 3:* Для каждого пользователя и для каждой модели постройте рекомендацию 5 лучших фильмов, используя функцию  Recomend(MarkList, N). Считая настоящие(UserRatings) оценки положительными в случае 5 или 4 и отрицательными во всех других случаях, посчитайте precision, recall и roc_auc.

*Задача 4:* Посчитайте NDCG для построенной выше рекомендации с бинарными оценками.

*Задача 5:* Посчитайте NDCG для построенной выше рекомендации, считая что $r_i=2$, при оценке 5, $r_i=1$, при оценке 4, $r_i=0$, в остальных случаях.  

*Задача 6:* Сравните модели по всем метрикам.

# Задача 1: Вычислите MAE, MSE и RMSE для этих моделей. Какая модель лучше?

In [16]:
import pandas as pd
tr_df = pd.DataFrame.from_dict(UserRatings)
pr_df = pd.DataFrame.from_dict(Mode1Ratings)

In [25]:
error_df = pd.wide_to_long((tr_df - pr_df).reset_index(),
                stubnames='User', i = "index", j = "id").reset_index()
error_df.User.mean()

0.13333333333333333

## MAE

In [29]:
def my_mae(true, predicted):
    '''
    :param true: real rating, dictionary
    :param predicted: model result, dictionaru
    :return: int, mean error
    '''
    tr_df = pd.DataFrame.from_dict(true)
    pr_df = pd.DataFrame.from_dict(predicted)
    error_df = pd.wide_to_long((tr_df - pr_df).reset_index(),
                               stubnames='User', i = "index", j = "id").reset_index()
    return(error_df.User.mean())

In [34]:
print("Model 1, MAE:")
print(my_mae(UserRatings, Model1Ratings))

Model 1, MAE:
0.13333333333333333


In [35]:
print("Model 2, MAE:")
print(my_mae(UserRatings, Model2Ratings))

Model 2, MAE:
-0.2


## MSE

In [38]:
def my_mse(true, predicted):
    '''
    :param true: real rating, dictionary
    :param predicted: model result, dictionaru
    :return: int, mean square error
    '''
    tr_df = pd.DataFrame.from_dict(true)
    pr_df = pd.DataFrame.from_dict(predicted)
    error_df = pd.wide_to_long((tr_df - pr_df).reset_index(),
                               stubnames='User', i = "index", j = "id").reset_index()
    print("MSE")
    return(error_df.User.pow(2).mean())

In [ ]:
print("Model 1")
print(my_mse(UserRatings, Model1Ratings))

In [ ]:
print("Model 2")
print(my_mse(UserRatings, Model2Ratings))

## RMSE

In [43]:
def my_rmse(true, predicted):
    '''
    :param true: real rating, dictionary
    :param predicted: model result, dictionaru
    :return: int, mean square error
    '''
    tr_df = pd.DataFrame.from_dict(true)
    pr_df = pd.DataFrame.from_dict(predicted)
    error_df = pd.wide_to_long((tr_df - pr_df).reset_index(),
                               stubnames='User', i = "index", j = "id").reset_index()
    print("RMSE")
    return(error_df.User.pow(2).pow(1./2).mean())

In [44]:
print("Model 1")
print(my_rmse(UserRatings, Model1Ratings))

Model 1
RMSE
0.8666666666666667


In [45]:
print("Model 2")
print(my_rmse(UserRatings, Model2Ratings))

Model 2
RMSE
0.9333333333333333


# *Задача 2:* Реализуйте функцию Recomend(MarkList, N) которая по списку оценок выводит лист из N id фильмов, соответствующих лучшим оценкам. При равенстве оценок лучше считается вильм с меньшим id. id фильма соответствует индекcу в MarkList.

In [47]:
import numpy as np

In [80]:
# arr = np.array(UserRatings["User1"])
df = pd.DataFrame(UserRatings["User1"], columns =['rating'])
df.sort_values(by = ["rating"], ascending=False).reset_index()['index'].head(100)

0    0
1    2
2    5
3    7
4    8
5    9
6    1
7    3
8    4
9    6
Name: index, dtype: int64

In [86]:
def Recomend(MarkList, N):
    df = pd.DataFrame(MarkList, columns =['rating'])
    return(df.sort_values(by = ["rating"], ascending=False)\
        .reset_index()['index']\
        .head(100))

In [88]:
Recomend(UserRatings["User1"], 100)

0    0
1    2
2    5
3    7
4    8
5    9
6    1
7    3
8    4
9    6
Name: index, dtype: int64

# *Задача 3:* Для каждого пользователя и для каждой модели постройте рекомендацию 5 лучших фильмов, используя функцию  Recomend(MarkList, N). Считая настоящие(UserRatings) оценки положительными в случае 5 или 4 и отрицательными во всех других случаях, посчитайте precision, recall и roc_auc.